# Import requirements

In [3]:
import sys
sys.path.append("../src")
import os
import pdb
import math
from scipy.io import loadmat
import torch
import numpy as np
import matplotlib.pyplot as plt
from kernelMatricesStore import KernelMatricesStore
from kernels import PeriodicKernel, ExponentialQuadraticKernel
from approxPosteriorForH import ApproxPosteriorForHForAllNeuronsAllTimes, ApproxPosteriorForHForAllNeuronsAssociatedTimes
from inducingPointsPrior import InducingPointsPrior
from expectedLogLikelihood import PointProcessExpectedLogLikelihood, PoissonExpectedLogLikelihood
from sparseVariationalLowerBound import SparseVariationalLowerBound
from klDivergence import KLDivergence
from sparseVariationalEM import SparseVariationalEM

# Load data and initial values

In [6]:
dataFilename = os.path.expanduser("data/demo_PointProcess.mat")
mat = loadmat(dataFilename)
nLatents = len(mat['Z0'])
nTrials = mat['Z0'][0,0].shape[2]
qMu = [torch.from_numpy(mat['q_mu0'][(0,i)]).type(torch.DoubleTensor).permute(2,0,1) for i in range(nLatents)]
qSVec = [torch.from_numpy(mat['q_sqrt0'][(0,i)]).type(torch.DoubleTensor).permute(2,0,1) for i in range(nLatents)]
qSDiag = [torch.from_numpy(mat['q_diag0'][(0,i)]).type(torch.DoubleTensor).permute(2,0,1) for i in range(nLatents)]
t = torch.from_numpy(mat['ttQuad']).type(torch.DoubleTensor).permute(2, 0, 1)
Z = [torch.from_numpy(mat['Z0'][(i,0)]).type(torch.DoubleTensor).permute(2,0,1) for i in range(nLatents)]
Y = [torch.from_numpy(mat['Y'][tr,0]).type(torch.DoubleTensor) for tr in range(nTrials)]
index = [torch.from_numpy(mat["index"][i,0][:,0]).type(torch.ByteTensor) for i in range(nTrials)]
C = torch.from_numpy(mat["C0"]).type(torch.DoubleTensor)
b = torch.from_numpy(mat["b0"]).type(torch.DoubleTensor).squeeze()
legQuadPoints = torch.from_numpy(mat['ttQuad']).type(torch.DoubleTensor).permute(2, 0, 1)
legQuadWeights = torch.from_numpy(mat['wwQuad']).type(torch.DoubleTensor).permute(2, 0, 1)
hermQuadPoints = torch.from_numpy(mat['xxHerm']).type(torch.DoubleTensor)
hermQuadWeights = torch.from_numpy(mat['wwHerm']).type(torch.DoubleTensor)
kernelNames = mat["kernelNames"]
hprs = mat["hprs0"]
testTimes = torch.from_numpy(mat['testTimes']).type(torch.DoubleTensor).squeeze()
trueLatents = [[torch.from_numpy(mat['trueLatents'][tr,k]).type(torch.DoubleTensor) for tr in range(nTrials)] for k in range(nLatents)]
linkFunction = torch.exp
kernels = [[None] for k in range(nLatents)]
for k in range(nLatents):
    if np.char.equal(kernelNames[0,k][0], 'PeriodicKernel'):
        kernels[k] = PeriodicKernel(scale=1.0, lengthScale=float(hprs[k,0][0]), period=float(hprs[k,0][1]))
    elif np.char.equal(kernelNames[0,k][0], 'rbfKernel'):
        kernels[k] = ExponentialQuadraticKernel(scale=1.0, lengthScale=float(hprs[k,0][0]))
    else:
        raise ValueError("Invalid kernel name: %s"%(kernelNames[k]))

# Create svGPFA Objects

In [7]:
qU = InducingPointsPrior(qMu=qMu, qSVec=qSVec, qSDiag=qSDiag, varRnk=torch.ones(3,dtype=torch.uint8))
kernelMatricesStore= KernelMatricesStore(kernels=kernels, Z=Z, t=t, Y=Y)
qH_allNeuronsAllTimes = ApproxPosteriorForHForAllNeuronsAllTimes(C=C, d=b, inducingPointsPrior=qU, kernelMatricesStore=kernelMatricesStore)
qH_allNeuronsAssociatedTimes = ApproxPosteriorForHForAllNeuronsAssociatedTimes(C=C, d=b, inducingPointsPrior=qU, kernelMatricesStore=kernelMatricesStore, neuronForSpikeIndex=index)
eLL = PointProcessExpectedLogLikelihood(approxPosteriorForHForAllNeuronsAllTimes=qH_allNeuronsAllTimes, approxPosteriorForHForAllNeuronsAssociatedTimes=qH_allNeuronsAssociatedTimes, hermQuadPoints=hermQuadPoints, hermQuadWeights=hermQuadWeights, legQuadPoints=legQuadPoints, legQuadWeights=legQuadWeights, linkFunction=linkFunction)
klDiv = KLDivergence(kernelMatricesStore=kernelMatricesStore, inducingPointsPrior=qU)
svLB = SparseVariationalLowerBound(eLL=eLL, klDiv=klDiv)
svEM = SparseVariationalEM(lowerBound=svLB, eLL=eLL, kernelMatricesStore=kernelMatricesStore)

# Maximize the Lower Bound

In [8]:
maxRes = svEM.maximize(emMaxNIter=50, eStepMaxNIter=50, mStepModelParamsMaxNIter=50, mStepKernelParamsMaxNIter=50, mStepKernelParamsLR=1e-5, mStepInducingPointsMaxNIter=50)

Iteration 00, E-Step start
Iteration: 0, negative lower bound: 6180.619714
Iteration: 10, negative lower bound: 5897.443745
Iteration: 20, negative lower bound: 5842.268679
Iteration: 30, negative lower bound: 5817.135911
Iteration: 40, negative lower bound: 5796.152578
Iteration 00, E-Step end: 5779.510134
Iteration 00, M-Step Model Params start
Iteration: 0, negative sum of expected log likelihood: 5465.384625
Iteration: 10, negative sum of expected log likelihood: 5445.059458
Iteration: 20, negative sum of expected log likelihood: 5425.724674
Iteration: 30, negative sum of expected log likelihood: 5407.401650
Iteration: 40, negative sum of expected log likelihood: 5390.050026
Iteration 00, M-Step Model Params end: 5375.205944
Iteration 00, M-Step Kernel Params start
Iteration: 0, negative lower bound: 5686.264706
Iteration: 10, negative lower bound: 5676.461573
Iteration: 20, negative lower bound: 5672.563180
Iteration: 30, negative lower bound: 5668.603493
Iteration: 40, negative l

Iteration: 10, negative lower bound: 4990.641029
Iteration: 20, negative lower bound: 4984.601488
Iteration: 30, negative lower bound: 4980.147806
Iteration: 40, negative lower bound: 4978.435107
Iteration 05, M-Step Inducing Points end: 4977.445523
Iteration 06, E-Step start
Iteration: 0, negative lower bound: 4977.422739
Iteration: 10, negative lower bound: 4972.703340
Iteration: 20, negative lower bound: 4967.741043
Iteration: 30, negative lower bound: 4963.714956
Iteration: 40, negative lower bound: 4960.060318
Iteration 06, E-Step end: 4956.943299
Iteration 06, M-Step Model Params start
Iteration: 0, negative sum of expected log likelihood: 4744.366621
Iteration: 10, negative sum of expected log likelihood: 4740.648481
Iteration: 20, negative sum of expected log likelihood: 4737.383679
Iteration: 30, negative sum of expected log likelihood: 4734.424494
Iteration: 40, negative sum of expected log likelihood: 4731.742221
Iteration 06, M-Step Model Params end: 4729.539990
Iteration 0

Iteration: 20, negative lower bound: 4833.060499
Iteration: 30, negative lower bound: 4833.007744
Iteration: 40, negative lower bound: 4832.993248
Iteration 11, M-Step Kernel Params end: 4832.974391
Iteration 11, M-Step Inducing Points start
Iteration: 0, negative lower bound: 4832.972503
Iteration: 10, negative lower bound: 4855.625708
Iteration: 20, negative lower bound: 4839.549761
Iteration: 30, negative lower bound: 4835.609548
Iteration: 40, negative lower bound: 4833.716208
Iteration 11, M-Step Inducing Points end: 4831.921551
Iteration 12, E-Step start
Iteration: 0, negative lower bound: 4831.878047
Iteration: 10, negative lower bound: 4830.715282
Iteration: 20, negative lower bound: 4828.082411
Iteration: 30, negative lower bound: 4826.312869
Iteration: 40, negative lower bound: 4824.875275
Iteration 12, E-Step end: 4823.641264
Iteration 12, M-Step Model Params start
Iteration: 0, negative sum of expected log likelihood: 4629.613545
Iteration: 10, negative sum of expected log 

Iteration: 40, negative sum of expected log likelihood: 4593.121497
Iteration 17, M-Step Model Params end: 4593.117855
Iteration 17, M-Step Kernel Params start
Iteration: 0, negative lower bound: 4780.315373
Iteration: 10, negative lower bound: 4780.692649
Iteration: 20, negative lower bound: 4780.351053
Iteration: 30, negative lower bound: 4780.275330
Iteration: 40, negative lower bound: 4780.267118
Iteration 17, M-Step Kernel Params end: 4780.252877
Iteration 17, M-Step Inducing Points start
Iteration: 0, negative lower bound: 4780.250230
Iteration: 10, negative lower bound: 4805.620121
Iteration: 20, negative lower bound: 4787.723419
Iteration: 30, negative lower bound: 4785.225140
Iteration: 40, negative lower bound: 4781.565569
Iteration 17, M-Step Inducing Points end: 4779.937473
Iteration 18, E-Step start
Iteration: 0, negative lower bound: 4779.807038
Iteration: 10, negative lower bound: 4779.156750
Iteration: 20, negative lower bound: 4777.030610
Iteration: 30, negative lower 

Iteration: 10, negative sum of expected log likelihood: 4570.231242
Iteration: 20, negative sum of expected log likelihood: 4570.225376
Iteration: 30, negative sum of expected log likelihood: 4570.223201
Iteration: 40, negative sum of expected log likelihood: 4570.222644
Iteration 23, M-Step Model Params end: 4570.222444
Iteration 23, M-Step Kernel Params start
Iteration: 0, negative lower bound: 4752.514017
Iteration: 10, negative lower bound: 4753.002120
Iteration: 20, negative lower bound: 4752.563736
Iteration: 30, negative lower bound: 4752.490832
Iteration: 40, negative lower bound: 4752.487607
Iteration 23, M-Step Kernel Params end: 4752.475064
Iteration 23, M-Step Inducing Points start
Iteration: 0, negative lower bound: 4752.471868
Iteration: 10, negative lower bound: 4767.068513
Iteration: 20, negative lower bound: 4757.210590
Iteration: 30, negative lower bound: 4753.995132
Iteration: 40, negative lower bound: 4752.730281
Iteration 23, M-Step Inducing Points end: 4751.837764

Iteration: 20, negative lower bound: 4735.421560
Iteration: 30, negative lower bound: 4734.829827
Iteration: 40, negative lower bound: 4734.505614
Iteration 29, E-Step end: 4734.240502
Iteration 29, M-Step Model Params start
Iteration: 0, negative sum of expected log likelihood: 4555.448615
Iteration: 10, negative sum of expected log likelihood: 4555.334977
Iteration: 20, negative sum of expected log likelihood: 4555.323314
Iteration: 30, negative sum of expected log likelihood: 4555.320017
Iteration: 40, negative sum of expected log likelihood: 4555.319075
Iteration 29, M-Step Model Params end: 4555.318695
Iteration 29, M-Step Kernel Params start
Iteration: 0, negative lower bound: 4734.081019
Iteration: 10, negative lower bound: 4734.624365
Iteration: 20, negative lower bound: 4734.168519
Iteration: 30, negative lower bound: 4734.060669
Iteration: 40, negative lower bound: 4734.056481
Iteration 29, M-Step Kernel Params end: 4734.051651
Iteration 29, M-Step Inducing Points start
Itera

Iteration: 30, negative lower bound: 4728.220252
Iteration: 40, negative lower bound: 4724.916762
Iteration 34, M-Step Inducing Points end: 4724.483888
Iteration 35, E-Step start
Iteration: 0, negative lower bound: 4724.202022
Iteration: 10, negative lower bound: 4724.665459
Iteration: 20, negative lower bound: 4723.469919
Iteration: 30, negative lower bound: 4723.075190
Iteration: 40, negative lower bound: 4722.916846
Iteration 35, E-Step end: 4722.791179
Iteration 35, M-Step Model Params start
Iteration: 0, negative sum of expected log likelihood: 4546.069401
Iteration: 10, negative sum of expected log likelihood: 4545.998249
Iteration: 20, negative sum of expected log likelihood: 4545.993766
Iteration: 30, negative sum of expected log likelihood: 4545.992634
Iteration: 40, negative sum of expected log likelihood: 4545.992023
Iteration 35, M-Step Model Params end: 4545.991806
Iteration 35, M-Step Kernel Params start
Iteration: 0, negative lower bound: 4722.699479
Iteration: 10, negat

Iteration: 40, negative lower bound: 4717.998755
Iteration 40, M-Step Kernel Params end: 4717.994380
Iteration 40, M-Step Inducing Points start
Iteration: 0, negative lower bound: 4717.987770
Iteration: 10, negative lower bound: 4742.331085
Iteration: 20, negative lower bound: 4726.280454
Iteration: 30, negative lower bound: 4725.245712
Iteration: 40, negative lower bound: 4721.436387
Iteration 40, M-Step Inducing Points end: 4718.867234
Iteration 41, E-Step start
Iteration: 0, negative lower bound: 4718.576676
Iteration: 10, negative lower bound: 4719.403476
Iteration: 20, negative lower bound: 4718.189445
Iteration: 30, negative lower bound: 4717.858082
Iteration: 40, negative lower bound: 4717.774149
Iteration 41, E-Step end: 4717.706231
Iteration 41, M-Step Model Params start
Iteration: 0, negative sum of expected log likelihood: 4541.973187
Iteration: 10, negative sum of expected log likelihood: 4541.955111
Iteration: 20, negative sum of expected log likelihood: 4541.953860
Iterat

Iteration 46, M-Step Model Params end: 4540.716227
Iteration 46, M-Step Kernel Params start
Iteration: 0, negative lower bound: 4714.179877
Iteration: 10, negative lower bound: 4714.787228
Iteration: 20, negative lower bound: 4714.288739
Iteration: 30, negative lower bound: 4714.153361
Iteration: 40, negative lower bound: 4714.151096
Iteration 46, M-Step Kernel Params end: 4714.150511
Iteration 46, M-Step Inducing Points start
Iteration: 0, negative lower bound: 4714.142674
Iteration: 10, negative lower bound: 4732.522242
Iteration: 20, negative lower bound: 4723.500193
Iteration: 30, negative lower bound: 4720.083083
Iteration: 40, negative lower bound: 4716.535132
Iteration 46, M-Step Inducing Points end: 4715.234607
Iteration 47, E-Step start
Iteration: 0, negative lower bound: 4714.850680
Iteration: 10, negative lower bound: 4715.301716
Iteration: 20, negative lower bound: 4714.133651
Iteration: 30, negative lower bound: 4713.868922
Iteration: 40, negative lower bound: 4713.785951


# Predict Latent Variables at New Times

In [9]:
qHMu, qHVar, qKMu, qKVar = qH_allNeuronsAllTimes.predict(testTimes=testTimes)

# Plot Latent Estimates for One Trial

In [10]:
Z = qH_allNeuronsAllTimes._kernelMatricesStore.getZ()
testTimesToPlot = testTimes.numpy()
trialToPlot = 0
f, axes = plt.subplots(nLatents, 1, sharex=True)
for k in range(nLatents):
    trueLatentToPlot = trueLatents[k][trialToPlot].numpy().squeeze()
    qKMuToPlot = qKMu[trialToPlot,:,k].numpy()
    errorToPlot = qKVar[trialToPlot,:,k].sqrt().numpy()
    axes[k].plot(testTimesToPlot, trueLatentToPlot, label="true", color="black")
    axes[k].plot(testTimesToPlot, qKMuToPlot, label="estimated", color="blue")
    axes[k].fill_between(testTimesToPlot, qKMuToPlot-errorToPlot, qKMuToPlot+errorToPlot, color="lightblue")
    for i in range(Z[k].shape[1]):
        axes[k].axvline(x=Z[k][trialToPlot,i, 0], color="red")
    axes[k].set_ylabel("Latent %d"%(k))
axes[-1].set_xlabel("Sample")
axes[-1].legend()
plt.xlim(left=np.min(testTimesToPlot)-1, right=np.max(testTimesToPlot)+1)
plt.show()

AttributeError: module 'matplotlib.colors' has no attribute 'to_rgba'